In [0]:
# vid2vid model to predict meteorological parameters based on ERA5 data

# Adapted from the pix2pix models developed here: 
# (1) https://www.tensorflow.org/tutorials/generative/pix2pix
# (2) https://machinelearningmastery.com/how-to-implement-pix2pix-gan-models-from-scratch-with-keras/

In [0]:
!pip install netCDF4
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install pyproj==1.9.6

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import compare_ssim as ssim
from scipy.interpolate import interp1d

from datetime import datetime, timedelta
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset

from google.colab import drive, files

import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Conv3D, Conv3DTranspose, Reshape, Flatten, \
  LeakyReLU, Activation, Dropout, Concatenate, BatchNormalization, \
  ConvLSTM2D, ZeroPadding3D

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal

from tensorflow.keras import metrics, losses

In [0]:
print(tf.__version__)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [0]:
drive.mount('/content/gdrive')

In [0]:
# Which field to forecast
field = 'geo'
# field = 'tp'
# field = 't2m'

In [0]:
# To run this, replace the path to the path of the downloaded ERA5 data (single level, pressure level, monthly means)
# from https://climate.copernicus.eu/climate-reanalysis for the domain specified in the paper
if field == 'geo':
  ncfid = Dataset('/content/gdrive/My Drive/Colab Notebooks/Era5Geopotential500.nc', mode='r') 
  ncfidc = Dataset('/content/gdrive/My Drive/Colab Notebooks/Era5Climatology500.nc', mode='r') 
elif field == 'tp':
  ncfid = Dataset('/content/gdrive/My Drive/Colab Notebooks/Era5Precipitation.nc', mode='r') 
  ncfidc = Dataset('/content/gdrive/My Drive/Colab Notebooks/Era5Climatology2m.nc', mode='r') 
elif field == 't2m':
  ncfid = Dataset('/content/gdrive/My Drive/Colab Notebooks/Era5T2m.nc', mode='r') 
  ncfidc = Dataset('/content/gdrive/My Drive/Colab Notebooks/Era5Climatology2m.nc', mode='r')

In [0]:
if field == 'geo':
  z = ncfid.variables['z'][:,:-1:2,:-1:2]
  z_c = ncfidc['z'][:,:-1:2,:-1:2]
elif field == 'tp':
  z = ncfid.variables['tp'][:,:-1:2,:-1:2]
  z_c = ncfidc['tp'][:,:-1:2,:-1:2]
elif field == 't2m':
  z = ncfid.variables['t2m'][:,:-1:2,:-1:2]
  z_c = ncfidc['t2m'][:,:-1:2,:-1:2]

In [0]:
# Get the auxiliary data
lon = ncfid.variables['longitude'][:-1:2]
lat = ncfid.variables['latitude'][:-1:2]
time = ncfid.variables['time'][:]

In [0]:
# Add fourth dimension to data
z = np.expand_dims(z, axis=3)
z_c = np.expand_dims(z_c, axis=3)

In [0]:
# Split into train and test data (0.8 -> 4 years of training, 1 year of testing)
test_split = int(0.8*len(z))

z_train = z[:test_split,]
z_test = z[test_split:,]

In [0]:
# Normalize the data
z_mean = z_train.mean()
z_std = z_train.std()

z_train = (z_train-z_mean)/z_std
z_test = (z_test-z_mean)/z_std
z_c = (z_c-z_mean)/z_std

In [0]:
# Resize the data (to speed up training)
z_train = tf.image.resize(z_train[:,], [128, 128], method=tf.image.ResizeMethod.BILINEAR)
z_test = tf.image.resize(z_test[:,], [128, 128], method=tf.image.ResizeMethod.BILINEAR)
z_c = tf.image.resize(z_c[:,], [128, 128], method=tf.image.ResizeMethod.BILINEAR)

In [0]:
# Resize the associated lat/long data
latInterp = interp1d(np.linspace(1, len(lat), len(lat)), lat)
lonInterp = interp1d(np.linspace(1, len(lon), len(lon)), lon)

lat = latInterp(np.linspace(1, len(lat), 128))
lon = lonInterp(np.linspace(1, len(lon), 128))

In [0]:
print(z_train.shape)

(11686, 128, 128, 1)


In [0]:
# Frames to predict
frames = 8

In [0]:
# Get data shape
samples, m, n, _ = z_train.shape

In [0]:
# Input data shape
input_shape = (frames, m, n, 1)

In [0]:
# define the encoder block
def define_encoder_block(layer_in, n_filters, stride, batchnorm=True):
	
  # weight initialization
	init = tf.random_normal_initializer(0., 0.02)
	
  # add downsampling layer
	g = Conv3D(n_filters, (4,4,4), strides=(stride,2,2), padding='same', 
            kernel_initializer=init, use_bias=False)(layer_in)
	
  # conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g
 
# define the decoder block
def decoder_block(layer_in, skip_in, n_filters, stride, dropout=True):
	
  # weight initialization
	init = tf.random_normal_initializer(0., 0.02)
	
  # add upsampling layer
	g = Conv3DTranspose(n_filters, (4,4,4), strides=(stride,2,2), padding='same', 
                     kernel_initializer=init, use_bias=False)(layer_in)
	
  # add batch normalization
	g = BatchNormalization()(g, training=True)
 
	# conditionally add dropout
	if dropout:
		g = Dropout(0.2)(g, training=True)
	
  # merge with skip connection
	g = Concatenate()([g, skip_in])
	
  # relu activation
	g = Activation('relu')(g)
	return g

In [0]:
# define the standalone generator model
def define_generator(image_shape):
	
  # weight initialization
	init = tf.random_normal_initializer(0., 0.02)
	
  # image input
	in_image = Input(shape=image_shape)
	
  # encoder model:
	e1 = define_encoder_block(in_image, 64, 2, batchnorm=False)
	e2 = define_encoder_block(e1, 128, 2)
	e3 = define_encoder_block(e2, 256, 2)
	e4 = define_encoder_block(e3, 256, 1)
	e5 = define_encoder_block(e4, 256, 1)
	b = define_encoder_block(e5, 256, 1)
 
	# bottleneck LSTM 
	b = ConvLSTM2D(256, (2,2), padding='same', return_sequences=True,
	               dropout=0.2, recurrent_dropout=0.2)(b, training=True)
	b = ConvLSTM2D(256, (2,2), padding='same', return_sequences=True,
	               dropout=0.2, recurrent_dropout=0.2)(b, training=True)
	
  # decoder model:
	d1 = decoder_block(b, e5, 256, 1)
	d2 = decoder_block(d1, e4, 256, 1)
	d3 = decoder_block(d2, e3, 256, 1, dropout=False)
	d4 = decoder_block(d3, e2, 128, 2, dropout=False)
	d5 = decoder_block(d4, e1, 64, 2, dropout=False)
	
  # output
	g = Conv3DTranspose(1, (4,4,4), strides=(2,2,2), padding='same', 
                     kernel_initializer=init)(d5)
	out_image = Activation('linear')(g)
 	
  # define model
	model = Model(in_image, out_image)
 
	return model

In [0]:
# Define generator
generator = define_generator(input_shape)
generator.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8, 128, 128, 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 4, 64, 64, 64 4096        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 4, 64, 64, 64 0           conv3d[0][0]                     
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 2, 32, 32, 12 524288      leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [0]:
def define_discriminator(in_shape=input_shape):

  # Weight initialization
  init = tf.random_normal_initializer(0., 0.02)

  inp_X = Input(shape=in_shape) # Input layer past data
  inp_Y = Input(shape=in_shape) # Input layer future data

  # Concatenate data
  h = Concatenate()([inp_X, inp_Y])

  h = define_encoder_block(h, 64, 2, batchnorm=False)
  h = define_encoder_block(h, 128, 1)
  h = define_encoder_block(h, 256, 1)

  h = ZeroPadding3D()(h)
  h = Conv3D(512, (4,4,4), strides=(1,1,1),
                                kernel_initializer=init,
                                use_bias=False)(h)
  h = BatchNormalization()(h)
  h = LeakyReLU(alpha=0.2)(h)

  h = ZeroPadding3D()(h)
  h = Conv3D(1, (4,4,4), strides=(1,1,1),
                                kernel_initializer=init)(h)
  outp = Activation('sigmoid')(h)

  model = Model([inp_X, inp_Y], outp)
  model.compile(loss='binary_crossentropy', 
                optimizer=Adam(lr=0.0002, beta_1=0.5),
                loss_weights=[0.5])
  
  return model

In [0]:
discriminator = define_discriminator(input_shape)
discriminator.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8, 128, 128, 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 8, 128, 128, 0                                            
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 8, 128, 128,  0           input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
conv3d_6 (Conv3D)               (None, 4, 64, 64, 64 8192        concatenate_5[0][0]        

In [0]:
# Now combine generator and discriminator into GAN model
def define_gan(g_model, d_model, image_shape):

	# Discriminator is not trainable
	d_model.trainable = False

	# Input video
	in_src = Input(shape=image_shape)
	
	# Generator input
	gen_out = g_model(in_src)
	
	# Input video and generator video are input to discriminator
	dis_out = d_model([in_src, gen_out])
 
	# GAN model
	model = Model(in_src, [dis_out, gen_out])
	
	# Compile with loss weights as in Isola et al. (2017)
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])
	
	return model

In [0]:
gan = define_gan(generator, discriminator, input_shape)
gan.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 8, 128, 128, 0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 8, 128, 128,  45635841    input_4[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 2, 14, 14, 1) 11054593    input_4[0][0]                    
                                                                 model[1][0]                      
Total params: 56,690,434
Trainable params: 45,631,617
Non-trainable params: 11,058,817
__________________________________________________________________________________________________


In [0]:
# Generate a batch of random samples, returns images and target
def generate_real_samples(data, frames, n_samples, patch_shape):

  samples, m, n, c = data.shape

  ind = np.random.randint(0, samples-2*frames, n_samples)
  X = np.zeros((n_samples, frames, m, n, c))
  Y = np.zeros((n_samples, frames, m, n, c))
  
  for i in range(0, n_samples):
    X[i,] = data[ind[i]:ind[i]+frames,]
    Y[i,] = data[ind[i]+frames:ind[i]+2*frames,]
  
  # True data so label = 1
  y = np.ones((n_samples, 2, patch_shape, patch_shape, 1))

  return [X, Y], y

In [0]:
def generate_fake_samples(g_model, samples, patch_shape):
	
	# generate fake instance
	X = g_model.predict(samples)
	
	# Fake data so label = 0
	y = np.zeros((len(X), 2, patch_shape, patch_shape, 1))
	
	return X, y

In [0]:
def train_gan(g_model, d_model, gan_model, dataset, frames,
          n_epochs=20, n_batch=16, n_patch=14):
	
	#E poch counter
	curr_epoch = 0

  # Calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	
  # Calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	
  # Manually enumerate epochs
	for i in range(1, n_steps):
		
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, frames, 
                                                     n_batch, n_patch)

		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)

    # update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		
    # update discriminator for fake samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		
    # update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		
    # summarize performance
		print('>Epoch %d, Batch %d, d1[%g] d2[%g] g[%g]' % (curr_epoch, i+1, d_loss1, d_loss2, g_loss))
	
		# Save model at each epoch to finally pick best one (change that to model callbacks in the future)
		if i%bat_per_epo == 0:
			print('Saving model at epoch {}'.format(curr_epoch))
			gan_model.save('gan_epoch' + str(curr_epoch) + '.h5')
			g_model.save('generator_epoch' + str(curr_epoch) + '.h5')
			d_model.save('discriminator_epoch' + str(curr_epoch) + '.h5')
			curr_epoch += 1

In [0]:
# Now train the GAN
train_gan(generator, discriminator, gan, z_train, frames)

In [0]:
# Change units of data for plotting purpose
# For precipitation: m to mm
if field == 'geo':
  scale = 1/9.81
  T0 = 0
# For geopential height: m to gpm
elif field == 'tp':
  scale = 1000
  T0 = 0
# For temperature: Kelvin to Celsius
elif field == 't2m':
  scale = 1
  T0 = -273.15

In [0]:
# Chooses a random sample from the test data set
def generate_sample(data, frames, input_shape):

  samples, m, n, c = data.shape
  ind = np.random.randint(frames, samples-frames, 1)

  X = np.zeros((1, frames, m, n, c))
  Y = np.zeros((1, frames, m, n, c))
  X[0,] = data[ind[0]-frames:ind[0],]
  Y[0,] = data[ind[0]:ind[0]+frames,]
  
  return ind, X, Y

# Chooses a specific example by index from the test data set
def generate_specific_sample(ind, data, frames, input_shape):

  samples, m, n, c = data.shape

  X = np.zeros((1, frames, m, n, c))
  Y = np.zeros((1, frames, m, n, c))
  X[0,] = data[ind[0]-frames:ind[0],]
  Y[0,] = data[ind[0]:ind[0]+frames,]
  
  return ind, X, Y

In [0]:
# Plotting functions for individual results
def plot_results(data, data_ref, ind, do_plot=False):

  samples, frames, _, _, _ = data.shape
  for i in range(frames):

    fig = plt.figure(figsize=(10,10))

    plt.subplot(131)
    m = Basemap(projection='cyl',
                llcrnrlon=-30,
                llcrnrlat=30.5, 
                urcrnrlon=29.5, 
                urcrnrlat=90, resolution='l')
    m.pcolormesh(lon, lat, scale*(z_std*data[0,i,:,:,0]+z_mean)+T0, 
                 latlon=True, cmap=plt.cm.coolwarm)
    m.colorbar(location='bottom')
    m.drawcoastlines(color='black')
    plt.title('vid2vid')

    plt.subplot(132)
    m = Basemap(projection='cyl',
                llcrnrlon=-30,
                llcrnrlat=30.5, 
                urcrnrlon=29.5, 
                urcrnrlat=90, resolution='l')
    m.pcolormesh(lon, lat, scale*(z_std*data_ref[0,i,:,:,0]+z_mean)+T0, 
                 latlon=True, cmap=plt.cm.coolwarm)
    m.colorbar(location='bottom')
    m.drawcoastlines(color='black')
    plt.title('Ground truth')

    plt.subplot(133)
    m = Basemap(projection='cyl',
                llcrnrlon=-30,
                llcrnrlat=30.5, 
                urcrnrlon=29.5, 
                urcrnrlat=90, resolution='l')
    if field == 'geo' or field == 't2m':
      rr = z_std*(data[0,i,:,:,0]-data_ref[0,i,:,:,0])/(z_std*data_ref[0,i,:,:,0]+ z_mean)
      # rr = scale*(z_std*(data[0,i,:,:,0]-data_ref[0,i,:,:,0]))
      plt.title('Relative error')
    elif field == 'tp':
      rr = scale*(z_std*(data[0,i,:,:,0]-data_ref[0,i,:,:,0]))
      plt.title('Absolute error')
    m.pcolormesh(lon, lat, rr, latlon=True, cmap=plt.cm.coolwarm)
    m.colorbar(location='bottom')
    m.drawcoastlines(color='black')
    
    if do_plot==True:
      plt.savefig("Run" + str(ind) + "_frame" + str(i) + ".png")
      files.download("Run" + str(ind) + "_frame" + str(i) + ".png")

    plt.show()

In [0]:
# Plotting function for ensemble predictions
def plot_ensemble(data, data_mean, data_std, ind, do_plot=False):

  samples, frames, _, _, _ = data.shape
  for i in range(frames):

    fig = plt.figure(figsize=(10,10))

    plt.subplot(131)
    m = Basemap(projection='cyl',
                llcrnrlon=-30,
                llcrnrlat=30.5, 
                urcrnrlon=29.5, 
                urcrnrlat=90, resolution='l')
    m.pcolormesh(lon, lat, scale*(z_std*data[0,i,:,:,0]+z_mean)+T0, 
                 latlon=True, cmap=plt.cm.coolwarm)
    m.colorbar(location='bottom')
    m.drawcoastlines(color='black')
    plt.title('Ground truth')

    plt.subplot(132)
    m = Basemap(projection='cyl',
                llcrnrlon=-30,
                llcrnrlat=30.5, 
                urcrnrlon=29.5, 
                urcrnrlat=90, resolution='l')
    m.pcolormesh(lon, lat, scale*(z_std*data_mean[0,i,:,:,0]+z_mean)+T0, 
                 latlon=True, cmap=plt.cm.coolwarm)
    m.colorbar(location='bottom')
    m.drawcoastlines(color='black')
    plt.title('Ensemble mean')

    plt.subplot(133)
    m = Basemap(projection='cyl',
                llcrnrlon=-30,
                llcrnrlat=30.5, 
                urcrnrlon=29.5, 
                urcrnrlat=90, resolution='l')
    m.pcolormesh(lon, lat, scale*data_std[0,i,:,:,0], latlon=True, cmap=plt.cm.coolwarm)
    m.colorbar(location='bottom')
    m.drawcoastlines(color='black')
    plt.title('Ensemble stdev')

    if do_plot==True:
      plt.savefig("Ensemble_run" + str(ind) + "_frame" + str(i) + ".png")
      files.download("Ensemble_run" + str(ind) + "_frame" + str(i) + ".png")

    plt.show()

In [0]:
# Test the generator by generating a random example
# ind, X, Y = generate_sample(z_test, frames, input_shape)
# Y_pred = generator.predict(X)

In [0]:
# The proper plotting and verification routines are in the accompanying
# vid2vidEra5_plotting_and_verification notebook. The following just serves to 
# quickly visualize some results of the trained models 

In [0]:
# Generate some specific examples 
# cases = 2208 (Lorenzo)
# cases = 1290 (Hailstorms)
ind, X, Y = generate_specific_sample(np.array([1290]), z_test, frames)
Y_pred = generator.predict(X)
print(datetime(2015, 1, 1, 0, 0) + +timedelta(hours=int(3*test_split)) + timedelta(hours=int(3*ind)))

In [0]:
plot_results(Y_pred, Y, ind)

In [0]:
runs = 100
Y_pred = generator(X)
Y_pred = generator.predict(X)
Y_prob = np.stack([generator(X, training=True) \
                     for sample in range(runs)])

Y_mean = Y_prob.mean(axis=0)
Y_std = (z_std*Y_prob+z_mean).std(axis=0)

plot_ensemble(Y, Y_mean, Y_std, ind)
print(datetime(2019, 1, 1) + timedelta(hours=int(3*ind)))

In [0]:
# Pick out particular locations for spaghetti ensemble

cities = ['bergen','rome','tromso','kalamata','reykjavik','vienna','london',
          'innsbruck', 'athens', 'lisbon', 'dublin','munich','milan']
lat_cities = [60.23, 41.9028, 69.6492, 37.0366, 64.1466, 48.208, 51.5074, 
              47.2692, 37.9838, 38.7223, 53.3498, 48.1351, 45.4642]
lon_cities = [5.195, 12.4964, 18.9553, 22.1144, -21.9426, 16.374, -0.128,
              11.4041, 23.7275, -9.1393, -6.2603, 11.5820, 9.1900]

# cities = ['innsbruck']
# lat_cities = [47.2692]
# lon_cities = [11.4041]

# Loop over cities to make ensemble plots
for i in range(len(cities)):
  lat_point = lat_cities[i]
  lon_point = lon_cities[i]
  city = cities[i]

  # Find associated values in prediction (here we pick the closest point to the city,
  # rather than interpolating to the city to avoid introducing an additional interpolation error)
  i_lat = np.abs(lat-lat_point) == np.min(np.abs(lat-lat_point))
  i_lon = np.abs(lon-lon_point) == np.min(np.abs(lon-lon_point))

  z_point_true = z_std*Y[0,:, i_lat, i_lon, 0].flatten()+z_mean
  z_point_mean = z_std*Y_mean[0,:, i_lat, i_lon, 0].flatten()+z_mean
  z_point_std = Y_std[0,:, i_lat, i_lon, 0].flatten()

  # Plot enembles
  plt.plot(np.linspace(3,3*frames,frames), scale*z_point_true+T0)
  plt.plot(np.linspace(3,3*frames,frames), scale*z_point_mean+T0,'k')
  for i in range(100):
    z_curr =  z_std*Y_prob[i,0,:, i_lat, i_lon, 0].flatten() + z_mean+T0
    #plt.plot(np.linspace(3,3*frames,frames), scale*z_curr, 'k', lw=0.1)
  plt.fill_between(np.linspace(3,3*frames,frames), 
                   scale*(z_point_mean-10*z_point_std)+T0, 
                   scale*(z_point_mean+10*z_point_std)+T0,
                   facecolor='k', alpha=0.5)
  plt.legend(['ERA5','ML_mean','ML_ens'])
  plt.grid()
  plt.xlim([3,3*frames])
  plt.xticks(ticks=np.linspace(3,3*frames,frames))
  plt.xlabel('Forecast time [h]')

  if field == 'geo':
    plt.ylabel('z [m]')
    plt.savefig('z_ensemble_' + city + '.png')
    files.download('z_ensemble_' + city + '.png')
  elif field == 'tp':
    plt.ylabel('tp [mm]')
    plt.savefig('tp_ensemble_' + city + '.png')
    files.download('tp_ensemble_' + city + '.png')
  elif field == 't2m':
    plt.ylabel('t2m [°C]')
    plt.savefig('t2m_ensemble_' + city + '.png')
    files.download('t2m_ensemble_' + city + '.png')
  
  plt.show()

In [0]:
# Save the models (to be used in plotting and verification routine)
generator.save('generator.h5')
discriminator.save('discriminator.h5')
gan.save('gan.h5')
files.download('generator.h5')
files.download('discriminator.h5')
files.download('gan.h5')